# centiBils: A unit for Investment Performance Management

## Abstract

Percentage Returns are the de-facto standard for Investment Performance
Measurement. Yet they are not very well suited for multi-period performance
evaluation. As an illustrative example, does the sequence (-30%, +40%) represent a
positive or negative overall return? We review some of the problems with using
percentage returns such as the lack of symmetry, lack of additivity and
differences between the arithmetic and geometric means. We then introduce a new
logarithmic unit modeled on the deciBel, tentatively called a centiBil, which
addresses these shortcomings. Finally we conclude by using the centiBil to
compare the performance of US Stocks, Bonds and Bills since 1927 and show how
their use eases cross-period comparisons.


## Agenda

 1. Returns vs Logeturns
      * Returns are not symmetric
      * Returns are not (longitudinally) additive
      * Arithmetic vs Geomtric mean
 2. Comparing existing logarithmic units
      * Neper
          * lacks interpretability
      * Bels (i.e. deciBels)
          * too large for use with investment returns
 3. centiBils
      * Definition
      * Standard Values
 4. Applications
      * S&P 500 Returns since 1900
      * Comparison of ALSI, ALBI and STEFI returns

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
%matplotlib inline

In [ ]:
def ret2cb(ret):
    return 100/np.log(2)*np.log1p(ret)

def cb2ret(cb):
    return np.expm1(cb/100*np.log(2))

## Definition

$$ 1 Bil = \ln_2 2 = \ln_2 (1+1) = \ln (1+100\%) / \ln 2$$

$$ r \implies 100 \ln_2(1+r) \textrm{ centiBils} = \frac{100}{\ln 2} \ln(1+r) \textrm{ cB} $$

## Standard Percentage Returns

In [ ]:
vals = np.array([-1, -0.50, -0.25, -0.20, -0.10, -0.01, -0.0001])

In [ ]:
std_rets = np.r_[vals, 0, -vals[::-1]]
standard_returns = pd.DataFrame({'Percentage Return':std_rets*100, 'centiBils Return':ret2cb(std_rets)})
standard_returns

In [ ]:
std_cbs = 100*np.r_[vals, 0, -vals[::-1]]
standard_centibils = pd.DataFrame({'centiBils Return':std_cbs, 'Percentage Return':cb2ret(std_cbs)*100})
standard_centibils

## Applications

In [ ]:
annual_periods = 261
N = 110*annual_periods
mu = 10/annual_periods
sigma = 20/np.sqrt(annual_periods)

In [ ]:
x = mu + sigma*np.random.standard_t(4, N)
idx = pd.date_range('1900-01-01', freq='B', periods=N)
cb = pd.Series(x, index=idx)

In [ ]:
cb.cumsum().plot(title='Cumulative Return in centiBils')

In [ ]:
r = cb2ret(cb)
cum_r = 100*np.expm1(np.log1p(r).cumsum())
cum_r.plot(title='Cumulative Return in percent')

In [ ]:
pd.rolling_sum(cb/10, window=10*annual_periods).plot()

## Distributions

### Daily

In [ ]:
sns.distplot(cb)

In [ ]:
sns.distplot(r)

### Grouped

In [ ]:
#grp_func = lambda idx: np.array(idx.strftime('%Y'), dtype=int)//10*10
grp_func = lambda idx: idx.strftime('%Y')
bins = None

In [ ]:
cb_grouped = cb.groupby(grp_func(cb.index)).sum()
sns.distplot(cb_grouped, bins=bins)
plt.title('Grouped centiBil returns')

In [ ]:
r_grouped = 100*r.groupby(grp_func(r.index)).agg(lambda rets: np.expm1(np.sum(np.log1p(rets))))
sns.distplot(r_grouped, bins=bins)
plt.title('Grouped percentage returns')

## S&P 500 Historic Performance (since 1871)

In [ ]:
data = pd.read_csv('../s-and-p-500/data/data.csv')
data.head()

In [ ]:
sp500 = data.iloc[:, :5].set_index('Date')
sp500 = sp500/sp500.iloc[0]
pd.concat([sp500.head(5), sp500.tail(5)])

In [ ]:
sp500.plot(figsize=(16,8))

In [ ]:
sp500.plot(figsize=(16,8), logy=True)

## US Asset Class Returns (since 1927)

In [ ]:
us_data = pd.read_excel('histretSP.xls', sheetname='Returns by year', skiprows=17, skip_footer=10)
pd.concat([us_data.head(), us_data.tail()])

In [ ]:
us = us_data[['Year', 'Stocks', 'T.Bonds', 'T.Bills']].set_index('Year')
us.loc[1927] = 100
us.sort_index(inplace=True)
us = us/us.iloc[0]
pd.concat([us.head(3), us.tail(3)])

In [ ]:
us.plot(figsize=(16,8))

In [ ]:
us.plot(figsize=(16,8), logy=True)

In [ ]:
np.log(us).plot(figsize=(16,8))

## python-nvd3

In [ ]:
import nvd3
#nvd3.ipynb.initialize_javascript(use_remote=True)
nvd3.ipynb.initialize_javascript()

In [ ]:
from nvd3 import lineChart
chart = lineChart(name='NVD3', x_is_date=False, width=1024)
for asset_class, series in idx2cb(us).iteritems():
    #x = ["{}-12-31".format(dt) for dt in series.index]
    x = [str(dt) for dt in series.index]
    chart.add_serie(name=asset_class, y=list(series.values), x=x)

chart.buildcontent()

HTML(chart.htmlcontent)